# Algorithms

Notes from reading *Grokking Algorithms* by Aditya Bhargava

# What is an algorithm?

An algorithm is a set of instructions for accomplishing a task.

There are often many ways to accomplish a task, and so we need to understand the trade-offs in order to know how best to accomplish a task in a given context.

## A motivating example: binary search

Imagine that we need to find an element in a sorted list. More specifically, we want to return the position of the desired element in that list, or `null` if it cannot be found.

**Simple search** might be the most obvious & basic attempt to solve that task. It starts at the first element, and moves through them one-by-one, in sequence, until it finds the right answer. If our desired value happens to be the first element, then we're in luck. If it isn't, and our list is long, then we might have to check a lot of values before we get our answer...

**Binary search** dramatically reduces the number of values we have to check before we find the position of our desired element. At each step it selects the mid-point of the remaining elements and establishes whether that location is too low or too high in order to determine which elements it can remove from consideration, and which elements are still contenders (remember, it's a sorted list). This process is repeated with each step until we find the target value (or exhaust the possibilities).

In [2]:
from typing import Union

def simple_search(sorted_elements: list, target_value: int) -> Union[int, None]:
    for index, element in enumerate(sorted_elements):
        if element == target_value:
            return index

In [3]:
sorted_list = [0, 1, 2, 3, 4, 5, 6, 7]

print(simple_search(sorted_list, 3))
print(simple_search(sorted_list, 6))
print(simple_search(sorted_list, 9))

3
6
None


In [20]:
def binary_search(sorted_elements: list, target_value: int) -> Union[int, None]:
    search_boundaries = {'low': 0, 'high': len(sorted_elements) - 1}
    guesses = 0
    while search_boundaries['low'] <= search_boundaries['high']:
        mid_point = int((search_boundaries['low'] + search_boundaries['high']) / 2)
        midpoint_value = sorted_elements[mid_point]
        guesses += 1
        if midpoint_value == target_value:
            print(f"{guesses} guesses")
            return mid_point
        if midpoint_value > target_value:
            search_boundaries['high'] = mid_point - 1
        else:
            search_boundaries['low'] = mid_point + 1
    return

binary_search([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 16)

5 guesses


20

## Selection Sort\*

# Recursion

A recursive recipe must contain three ingredients:

1. Stopping criteria
2. A first step (gets the ball rolling)
3. A repetitive component (that will lead us to the stopping criteria) ie. a function that calls itself.

Recursion does not improve computational performance, but may be faster to write. It should be used when it helps to make the solution easier to intuit.

## Structure

In order to avoid infinite loops of recursion, each recursive method needs to have two components:
1. A base case: catches the stopping criteria & breaks the loop
2. A recursive case: takes the recursion a layer deeper

## The call stack

The *stack* data structure acts like a LIFO inventory: new items go to the top of the pile as they're added and we remove those most recent items from the pile first as we work through it.

The **call stack** is a stack data structure that our computers use to keep track of function calls that are WIP:
- New function calls are added to the stack (memory is allocated to that call, and variables in that scope are saved to that memory)
- Those function calls are removed from the stack as they return their results

Because recursion involves calling the same function many times, with a chain of dependency between each call, it can lead to a lot of function calls being added to the call stack. With enough calls, or large state stored for each call, recursion can exhaust a machine's memory.

When this occurs, there are two options:
1. Use a loop instead
2. Use tail recursion (which only some languages support)

# Divide & Conquer

**Divide & Conquer** is a general technique for solving problems that uses recursion.

1. Identify simple conditions in which the problem can be solved.
2. Break the problem down so that you are only trying to solve it for increasingly small / simple inputs; continue until you reach a situation where you only need to deal with the simple conditions above. 

> NB. When working with recursion on problems involving arrays, the base / simple case is often a array of length 0 or 1.

Preffering the recursive approach over loops is typical of **functional programming** - Haskell doesn't even have loops!

The Binary Search algorithm that we saw earlier is also a type of D&C solution which can be coded using recursion. See the re-write of our earlier function below:

In [44]:
def recursive_binary_search(arr: list, target: int, low_idx: int, high_idx: int, guesses: int = 0) -> Union[int, None]:
    if not low_idx < high_idx:
        return None
    guesses += 1
    # Base case
    mid_point = int(low_idx + (high_idx - low_idx) / 2)
    print('mid: ', mid_point)
    midpoint_value = arr[mid_point]
    if midpoint_value == target:
        print(f"{guesses} guesses")
        return mid_point
    elif midpoint_value > target:
        return recursive_binary_search(arr, target, low_idx, mid_point-1, guesses=guesses)
    else:
        return recursive_binary_search(arr, target, mid_point+1, high_idx, guesses=guesses)

In [51]:
arr = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
recursive_binary_search(arr, 4, 0, len(arr))

mid:  10
mid:  4
2 guesses


4

## Quicksort

Quicksort - as the name suggests - is a sorting algorithm that uses D&C.

We'll work through it using the example of sorting an array:

**1. Identify simple conditions in which the problem can be solved.**

The simplest situation we can hope to work with is a case where the array is either empty, or only has one element: in this scenario, we can just return the array as it is because there's no sorting to be done!

This could look something like:
```python
def quicksort(arr):
    if len(arr) < 2:
        return arr
```

The next simplest situation we can have is an array with 2 elements. In this scenario, if the first element is larger than the second, then swap them around and return.

In [52]:
def quicksort(arr):
    if len(arr) < 2:
        return arr
    if len(arr) == 2:
        return arr if arr[0] < arr[1] else arr[::-1]
    
quicksort([2,1])

[1, 2]

**2. Then break the problem down until you reach those conditions.**

And now we need to think through how we'd approach longer arrays. With 3 elements, we can no longer do the direct comparison above. We need to Divide & Conquer... How could we break up the process so that we end up only having to deal with many instances of the base case?

One approach could be to:
- Select an element; we'll call our selected element the ***pivot***
- Work through & throw all other elements into one of two buckets (leaving three in total, once the pivot is included): those less than the pivot, and those greater than the pivot
- For each of the non-pivot buckets, choose a new pivot and repeat
- Eventually, each non-pivot bucket will simplify to an array of 2 or fewer elements, and from these we can build a complete array of sorted elements!

In [7]:
lesser = []
greater = []
arr = [5,4,3,2,1]
pivot = 3

for elem in arr:
    lesser.append(elem) if elem <= pivot else greater.append(elem)
    
print(lesser)
print(greater)

[3, 2, 1]
[5, 4]


In [11]:
def quicksort(arr: list):
    if len(arr) < 2:
        return arr
    if len(arr) == 2:
        return arr if arr[0] < arr[1] else arr[::-1]
    
    lesser = []
    greater = []
    pivot = arr.pop()
    for elem in arr:
        lesser.append(elem) if elem <= pivot else greater.append(elem)
    
    return quicksort(lesser) + [pivot] + quicksort(greater)
    

assert quicksort([1, 2]) == [1, 2]
assert quicksort([1, 2, 3]) == [1, 2, 3], f"returns {quicksort([1,2,3])}"
assert quicksort([7, 6, 5, 4, 2, 3, 1]) == [1, 2, 3, 4, 5, 6, 7], f"returns {quicksort([1,2,3,4,5,6,7])}"

### A note on complexity

The speed with which quicksort sorts an array depends on which pivots we choose:
- In the worst case, the complexity of running quicksort is $O(n^{2})$ (ie. as bad as **selection sort**)
- In the average case, the complexity of running quicksort is $O(nlog(n))$

The worst case occurs if we manage to always select our pivot such that all remaining elements lie in just one partition (ie. all are lesser than, or greater than, the pivot). This results in the worst case, because we're effectively removing one item from the array at a time, rather than breaking it into many chunks. This would occur when the array passed in is already sorted and we select the first element as pivot on each split.

To avoid the worst case, and aim for the average case, we usually select the pivot element by random each split.

In contrast, the best case occurs if each pivot is bang in the middle of the array, so the partitions are split equally - which speeds up how quickly we break all branches down to a base case scenario. In this case, the runtime complexity is $O(log(n))$.

The average case complexity is equal to the best case complexity! (the runtime will be slower because we're unlikely to get exactly the ideal splits every time, but it scales with $n$ in the same order of complexity).

### Parallelism?

Looking at the shape of the quicksort method we've written above, it's interesting to see that not only are we using recursion, we're using recursion twice (in that last line).
The fact that we're breaking the problem down into many branching pieces more quickly (by calling it twice, rather than having a single chain of many dependent calls) suggests that there may be more opportunity for parallel processing to help us here?

# Merge sort



# Avg-case Vs Worst case

In all the complexity notation, there's a hidden constant; figuratively and literally...

In all cases, the runtime of an algorithm depends not just on $n$, but also $c$: the time taken to compute the required operations for each element of $n$.
For some algorithms, the order of complexity may be higher than others, but $c$ may be small enough that it's still faster to use for the size of $n$ you're dealing with.

# Recap


| Algorithm     | Worst-case runtime complexity      |
|:--------------|:-------------:|
| Binary search |  |
| Selection sort|     $O(n^{n}$     |
| Quicksort     |          |
| Merge sort    |          |

# Resources

[Eliana Lopez's crib sheet](https://github.com/elianalopez/Data-Structures-and-Algorithms-Notes-with-Python)
[Complexity of Python Operations](https://www.ics.uci.edu/~pattis/ICS-33/lectures/complexitypython.txt)